# Reconhecimento de Entidades Nomeadas

Modelo: LSTM tradicional

O corpus e os seus detalhes podem ser encontrado aqui:

https://huggingface.co/datasets/conll2003

In [ ]:
!pip install datasets

In [20]:
from datasets import load_dataset
import pandas as pd
import torch
import torch.nn as nn
import numpy as np

In [21]:
class LSTM_NER(nn.Module):

    def __init__(self,embedding_dim,hidden_dim,vocab_size,num_classes):
        super(LSTM_NER,self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(vocab_size,embedding_dim) #sem pre-treinamento
        self.lstm = nn.LSTM(embedding_dim,hidden_dim)
        self.fcl = nn.Linear(hidden_dim,num_classes)

    def forward(self,sentence):
        word_embeddings = self.embedding(sentence) # vetor dimensoes: 1 x embedding_dim
        hidden_state,cell_state = self.lstm(word_embeddings) # demanda mais processamento
        tag_space = self.fcl(hidden_state) # demanda mais processamento
        return tag_space

Carrega o corpus CONLL, apenas o subset de treinamento, para a memória

In [22]:
train_dataset = load_dataset('conll2003',split='train')

In [23]:
df_train = pd.DataFrame(train_dataset)

Visão do corpus

In [24]:
df_train

,id,tokens,pos_tags,chunk_tags,ner_tags
0,0,"[EU, rejects, German, call, to, boycott, Briti...","[22, 42, 16, 21, 35, 37, 16, 21, 7]","[11, 21, 11, 12, 21, 22, 11, 12, 0]","[3, 0, 7, 0, 0, 0, 7, 0, 0]"
1,1,"[Peter, Blackburn]","[22, 22]","[11, 12]","[1, 2]"
2,2,"[BRUSSELS, 1996-08-22]","[22, 11]","[11, 12]","[5, 0]"
3,3,"[The, European, Commission, said, on, Thursday...","[12, 22, 22, 38, 15, 22, 28, 38, 15, 16, 21, 3...","[11, 12, 12, 21, 13, 11, 11, 21, 13, 11, 12, 1...","[0, 3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, ..."
4,4,"[Germany, 's, representative, to, the, Europea...","[22, 27, 21, 35, 12, 22, 22, 27, 16, 21, 22, 2...","[11, 11, 12, 13, 11, 12, 12, 11, 12, 12, 12, 1...","[5, 0, 0, 0, 0, 3, 4, 0, 0, 0, 1, 2, 0, 0, 0, ..."
...,...,...,...,...,...
14036,14036,"[on, Friday, :]","[15, 22, 8]","[13, 11, 0]","[0, 0, 0]"
14037,14037,"[Division, two]","[21, 11]","[11, 12]","[0, 0]"
14038,14038,"[Plymouth, 2, Preston, 1]","[21, 11, 22, 11]","[11, 12, 12, 12]","[3, 0, 3, 0]"
14039,14039,"[Division, three]","[21, 11]","[11, 12]","[0, 0]"


Separa as palavras já tokenizadas e as tags das entidades em duas estruturas separadas

In [25]:
corpus = train_dataset['tokens']
entities = train_dataset['ner_tags']

Uma pequena amostra (1a sentença) do corpus CONLL

In [26]:
for i in range(0,len(corpus[0])):
    print(f'Word: {corpus[0][i]} || Entity {entities[0][i]}')

Word: EU || Entity 3
Word: rejects || Entity 0
Word: German || Entity 7
Word: call || Entity 0
Word: to || Entity 0
Word: boycott || Entity 0
Word: British || Entity 7
Word: lamb || Entity 0
Word: . || Entity 0


Construção do vocabulário a partir dos textos de treinamento.

Devido ao espaço ocupado em memória ser alto para os padrões do Colab (pode resultar em crash na sessão), teremos que limitar o processamento apenas para as 200 primeiras instâncias (exemplos)

In [27]:
word2index = {'OOV' : 0} #out of vocabulary - OOV eh notacao comum em PLN
for i in range(200):#len(corpus)): #restricao a 200 sentenca, devido ao tempo da aula
    for j in range(len(corpus[i])):
        word = corpus[i][j]
        if corpus[i][j] not in word2index:
            word2index[corpus[i][j]] = len(word2index)

In [28]:
vocab_size = len(word2index)
vocab_size

1279

Temos nove tipos de entidades no corpus CONLL

In [29]:
num_classes = 9

Definindo alguns hiperparâmetros

In [30]:
embedding_dim = 128
hidden_dim = 128
learning_rate = 1e-4
num_epochs = 10

Instancia o objeto da classe LSTM_NER, que será o nosso modelo de reconhecimento de entidades nomeadas

In [31]:
model_ner = LSTM_NER(embedding_dim,hidden_dim,vocab_size,num_classes)

Podemos ver abaixo como é a arquitetura do nosso modelo:

In [32]:
model_ner

LSTM_NER(
  (embedding): Embedding(1279, 128)
  (lstm): LSTM(128, 128)
  (fcl): Linear(in_features=128, out_features=9, bias=True)
)

Declaramos a função loss do tipo Entropia Cruzada (considera implicitamente os labels como sendo one hot encoding) e o otimizador Adam para efetuar o backpropagation

In [33]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_ner.parameters(),lr=learning_rate)

Processo de treinamento da rede. Aqui, por motivos de simplicidade, o batch_size é igual a 1, isto é, uma cada batch contém uma única sentença.


In [34]:
for epoch in range(num_epochs):

    total_loss = 0

    for i in range(200):#range(len(train_dataset)): #batch_size = 1

        model_ner.train() # ativa o estado de treinamento do modelo

        sentence = train_dataset['tokens'][i]
        entity_tags = train_dataset['ner_tags'][i]

        model_ner.zero_grad() # zera os gradientes

        sentence_int = torch.tensor([word2index[word] for word in sentence],dtype=torch.long)
        entity_tags = torch.tensor([tag for tag in entity_tags],dtype=torch.long)

        # passo forward
        entity_predictions = model_ner(sentence_int)

        loss = loss_function(entity_predictions,entity_tags)
        # prepara para backpropagation: calculo dos gradientes
        loss.backward()

        optimizer.step() # otimizador, atualize os parametros do modelo

        total_loss += loss.item() #pega a loss calculada

    print(f'Epoch {epoch+1}  ======== Loss: {total_loss:.5f}')

Epoch 1  ======== Loss: 410.85097
Epoch 2  ======== Loss: 239.13361
Epoch 3  ======== Loss: 189.97863
Epoch 4  ======== Loss: 173.00689
Epoch 5  ======== Loss: 162.14282
Epoch 6  ======== Loss: 154.01375
Epoch 7  ======== Loss: 147.22686
Epoch 8  ======== Loss: 141.09146
Epoch 9  ======== Loss: 135.23812
Epoch 10  ======== Loss: 129.46865


Rotulando cada palavra de acordo com o reconhecimento de entidades nomeadas para uma sentença de teste:

In [35]:
test_sentence = "The United States started to struggle with John Kayne"

torch.no_grad() #nao havera calculo de gradiente, pois sao dados de teste, nao requer retreinar o modelo

test_int = []
for word in test_sentence.split():
    if word in word2index:
        word_int = word2index[word]
    else:
        word_int = word2index['OOV']
    test_int.append(word_int)

Pode-se ver que as predições indicam que cada palavra foi rotulada como sendo pertencentes a nenhuma das entidades mais específicas (pessoais, locais, organizações ou mix), sendo todas rotuladas como "O".

In [36]:
test_tensor = torch.tensor(test_int,dtype=torch.long)

tag_scores = model_ner(test_tensor)

_,predicted = torch.max(tag_scores,1)

print(predicted)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0])
